#### Install packages

In [154]:
from selenium import webdriver

# pip install pandas in command line
import pandas as pd

# import packages to set wait time for page to load before running Selenium code
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import time 
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains

# packages to download images
import urllib
import urllib.request

# regex
import re

# sorting
import string

#### Get urls for all regions

In [142]:
# set webdriver to Chrome
driver = webdriver.Chrome()

# website start URL
driver.get("https://www.craigslist.org/about/sites")

# pull all a tags with href attribute 
regions = driver.find_elements_by_xpath('//a[@href]')

# append hrefs without www in url to regions_list
regions_list = []
for region in regions:
    href = region.get_attribute('href')
    if bool(re.search('www', href)) == True:
        pass
    
    elif bool(re.search('https', href)) == False:
        pass
    
    elif bool(re.search('forum', href)) == True:
        pass
    
    else:
        regions_list.append(href)

# close webdriver
driver.close()

In [143]:
# append query with by owner, mirror search, in furniture to each region url
regions_list_search = [i + "search/fuo?query=mirror&sort=rel" for i in regions_list]

#### Get urls for all posts

In [200]:
# set webdriver to Chrome
driver = webdriver.Chrome()

count = 0
post_list = []
for i in regions_list_search:

    # get first region query
    driver.get(i)
    
    # look for seach bar to enter return
    try:
        element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="query"]')))
    except: 
        pass
    
    # enter return on search bar 
    try:
        driver.find_element_by_xpath('//*[@id="query"]').send_keys(Keys.RETURN)
    except:
        pass
    
    try:
        # pull all a tags with href attribute 
        posts = driver.find_elements_by_xpath('//a[@href]')
        
        # append hrefs to post_list
        time.sleep(.5)
        for post in posts:
            href = post.get_attribute('href')
            post_list.append(href)
    
    except:
        pass

#close webdriver 
driver.close()

# eliminate duplicates and sort urls from each region
post_list_set = sorted(set(post_list))

In [212]:
# # store first list - hashed out as name needs to be altered or it will save over existing file
# post_series = pd.Series(post_list_set)
# post_series.to_csv('post_list.csv')


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  This is separate from the ipykernel package so we can avoid doing imports until


#### Filter out urls that aren't for individual listings

In [216]:
post_list_filtered = []
for i in post_list_set:
    
    # eliminate urls with "www" and "search"
    if bool(re.search(('www | search'), i)) == True:
        pass

    # eliminate urls not in furniture section
    elif bool(re.search(('fuo'), i)) == False:
        pass

    # eliminate urls that aren't html page
    elif bool(re.search('html', i)) == False:
        pass

    else:
        post_list_filtered.append(i)

In [221]:
# number of listings
len(post_list_filtered)

22052

#### Pull images from each listing url

In [232]:
# set webdriver to Chrome
driver = webdriver.Chrome()

for i in post_list_filtered:
    
    # get region
    region = re.split('//|\.', i)[1]
    
    # website start URL for first url in regions_list
    driver.get(i)
    
    # look for post_id number
    try:
        element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '/html/body/section/section/section/div[2]/p[1]')))
    except: 
        pass
    
    # get post id number
    post_id = driver.find_element_by_xpath('/html/body/section/section/section/div[2]/p[1]')
    post_id_num = re.split(': ', post_id.text)[1]
    
    
    try:
        # find all img tags
        images = driver.find_elements_by_tag_name('img')

        # download 600x450 jpgs, convert 50x50c jps to 600x450, eliminate dups, download all
        image_num = 0
        srcs = []
        for image in images:

            image_num+=1

            src = image.get_attribute('src')

            if bool(re.search('600x450', src)) == True:
                if str(src) in srcs:
                    pass

                else:
                    data = urllib.request.urlretrieve(src, region + "_" + post_id_num + "_" + str(image_num) + '.jpg')
                    srcs.append(src)


            elif bool(re.search('images\.', src)) == True:
                src2 = re.sub("50x50c", "600x450", src)

                if str(src2) in srcs:
                    pass

                else:
                    data = urllib.request.urlretrieve(src2, region + "_" + post_id_num + "_" + str(image_num) + '.jpg')

            else:
                pass
            
    except:
        pass

#close webdriver 
driver.close()

KeyboardInterrupt: 